In [13]:
%load_ext autoreload
%autoreload 2
import time
from tweepy import TweepError
from requests.exceptions import Timeout, ConnectionError
import os
import tweepy
import json
import requests
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# THE CELL BELOW CONTAINS THE FILENAME CONSTANTS THAT NEED TO BE CHANGED

In [14]:
'''
    SOME QUICK HOW TO!
    
    To make this work, make sure you:
        - Have created an output folder - python won't do it if you haven't.
        - Have a lot of space and time, because this will probably take a long, long time.
        - Maybe have some kind of check for what happens after 15 minutes of pulling data,
            I have not tested it for that limit.
'''

# This file should be the one with the tweets as jsons in it.
TWEET_ID_FILE = "C:/Users/mfren/Desktop/maria_extras.json"

# The downloaded images should go into a folder.
OUTPUT_DIR = "C:/Users/mfren/Desktop/maria_images/"
retry_group = []

In [15]:
def exists(path):
  r = requests.head(path)
  return r.status_code == requests.codes.ok

In [16]:
with open(TWEET_ID_FILE) as f:
    all_content_as_string = f.read()

In [17]:
# split the file into a list of strings.
list_of_dicts = all_content_as_string.splitlines()

# turn each string into a json object
list_of_jsons = [json.loads(string) for string in list_of_dicts]

In [18]:
# Go through the whole list of tweets and download the images.

image_count = 0
non_image_count = 0

for i in range(0, len(list_of_jsons)):

    # If there is media attached to the tweet.
    if 'extended_entities' in list_of_jsons[i]:
        # Make a temp (a little cleaner than getting it constantly)
        currentJson = dict(list_of_jsons[i])

        if len(list_of_jsons[i]['extended_entities']['media']) > 1: # Only iterate through and get the additional pics if
            # the tweet had more than one image attached.

            # Ok, iterate through each link in the media list - excluding the first one because we already have it.
            for j in range(1, len(currentJson['extended_entities']['media'])):

                # Assign the URL for the media-piece.
                currentUrl = currentJson['extended_entities']['media'][j]['media_url_https']

                # Check if the picture URL works and if the photo is indeed, a photo
                try:
                    if (exists(currentUrl) and currentJson['extended_entities']['media'][j]['type'] == 'photo'):
                        # Do the raw stream magic (Who knows how this works)
                        url_raw = requests.get(currentUrl, stream=True)
                        url_raw.raw.decode_content = True

                        # NAMING SCHEME: tweetID + userID + index of img
                        with open(OUTPUT_DIR + str(currentJson['id']) + '_' +
                                  str(currentJson['user']['id']) + "_" + str(j) + ".jpg", 'wb+') as fin:
                            shutil.copyfileobj(url_raw.raw, fin)

                        # Remove the image url response object - because you should.
                        del url_raw

                        image_count += 1
                except requests.exceptions.RequestException as e:
                    retry_group.append(currentJson)
    else:
        non_image_count += 1
    if (i % 500 == 0):
        print("Finished tweet number ", i)


# Repeat the same process for the retry group until the retry list is empty:
while len(retry_group) != 0:
    print("Running through the retry group now... ", len(retry_group), " items in the retry group: ")

    # If there is media attached to the tweet.
    if 'extended_entities' in retry_group[0]:
        # Make a temp (a little cleaner than getting it constantly)
        currentJson = dict(retry_group[0])
        retry_group.pop(0)  # Remove the current json from the retry list

        if len(retry_group[0]['extended_entities']['media']) > 1:  # Only iterate through and get the additional pics if
        # the tweet had more than one image attached.

            # Ok, iterate through each link in the media list.
            for j in range(1, len(currentJson['extended_entities']['media'])):

                # Assign the URL for the media-piece.
                currentUrl = currentJson['extended_entities']['media'][j]['media_url_https']

                # Check if the picture URL works and if the photo is indeed, a photo
                try:
                    if (exists(currentUrl) and currentJson['extended_entities']['media'][j]['type'] == 'photo'):
                        # Do the raw stream magic (Who knows how this works)
                        url_raw = requests.get(currentUrl, stream=True)
                        url_raw.raw.decode_content = True

                        # NAMING SCHEME: tweetID + userID + index of img
                        with open(OUTPUT_DIR + str(currentJson['id']) + '_' +
                                  str(currentJson['user']['id']) + "_" + str(j) + ".jpg", 'wb+') as fin:
                            shutil.copyfileobj(url_raw.raw, fin)

                        # Remove the image url response object - because you should.
                        del url_raw

                        image_count += 1
                except requests.exceptions.RequestException as e:  # If another error occurs, add it back to the retry list
                    retry_group.append(currentJson)
        else:
            non_image_count += 1


# So you know once it is done, + some S T A T I S T I C S
print("Found and downloaded", image_count, " images and", non_image_count, " other tweets. Total: ", i)

Finished tweet number  0
Finished tweet number  500
Finished tweet number  1000
Finished tweet number  1500
Finished tweet number  2000
Finished tweet number  2500
Finished tweet number  3000
Finished tweet number  3500
Finished tweet number  4000
Finished tweet number  4500
Finished tweet number  5000
Finished tweet number  5500
Finished tweet number  6000
Finished tweet number  6500
Finished tweet number  7000
Finished tweet number  7500
Finished tweet number  8000
Finished tweet number  8500
Finished tweet number  9000
Finished tweet number  9500
Finished tweet number  10000
Finished tweet number  10500
Finished tweet number  11000
Finished tweet number  11500
Finished tweet number  12000
Finished tweet number  12500
Finished tweet number  13000
Finished tweet number  13500
Finished tweet number  14000
Finished tweet number  14500
Finished tweet number  15000
Finished tweet number  15500
Finished tweet number  16000
Finished tweet number  16500
Finished tweet number  17000
Finished 